### Emma Klemets, 260775167   
All my work, code, output and written answers are in this notebook.

In [1]:
#for interactive plots - not what I want

# %matplotlib
# %matplotlib inline
# %matplotlib notebook

In [2]:
import numpy as np
import matplotlib

#for pop out plots - windows
matplotlib.use('Qt5Agg')
import matplotlib.pyplot as plt
import time
import matplotlib.cm as cm
from mpl_toolkits.mplot3d import Axes3D

Make a 3-D nbody code that calculates the forces by computing the potential, where the potential is found by convolving the density with the (softened) potential from a single particle. The acceleration is then found by taking the gradient of the potential. You will probably wish to use a leapfrog solver with fixed timestep.

In [3]:
"""
calculates the forces by computing the potential

potential is found by convolving the density with the (softened) potential from a single particle. 
phi = rho x V_1

The acceleration is then found by taking the gradient of the potential.
acc = grad phi


calculate the potential using the Green's function!
So you can use the Green's function of the laplacian : -1/(4*pi*r) where r = sqrt(x^2 + y^2 + z^2)
Since (taking 4*pi*G=1): laplacian(phi) = rho ,

solve phi = ifft(fft(G_laplacian) * fft(rho))  where G_laplacian is just the Green's function of the laplacian

- how does it not matter which particle for rho?
- what is the r in G's relative to - should it be between two particles? 
"""
"""
To do later:
- use some sort of gif thing to save the animation things
    https://ffmpeg.org/
- add in G value for gravity (right now just 1 I guess)

With non-p I basicly have no particles by the end
"""
x=1

In [11]:
class Particles:
    def __init__(self,x,v,m, gridSize, grid_dx=1, BC_per=False):
        #set up our grid - gridSize should be [x, y, z]
        x_g = np.arange(0, gridSize[0], grid_dx)
        y_g = np.arange(0, gridSize[1], grid_dx)
        z_g = np.arange(0, gridSize[2], grid_dx)
        self.grid = np.meshgrid(x_g, y_g, z_g)
        self.grid_shape = np.array(gridSize)
        self.dt = grid_dx
        self.f_grid = np.meshgrid(self.grid[0][0]*0, self.grid[0][0]*0, self.grid[0][0]*0)
        
        self.BC = BC_per #true for periodic BC, false for not
        self.potential = np.meshgrid(self.grid[0][0]*0)
        
        self.x=x.copy()
        self.v=v.copy()
        try:
            self.m=m.copy()
        except:
            self.m=m
            
        self.f=np.empty(x.shape)
        self.a=np.empty(x.shape)
        self.n=self.x.shape[0]
        
        #maybe save energy?
        
    ############ methods for calculations for each particle (as seen in class) #############
    def r(self):
        return np.sqrt(np.sum(self.x**2, axis=1))
        
    def get_forces(self, soft=0.01, do_pot=False):
        self.f[:]=0
        for i in range(self.n):
            for j in range(self.n):
                if i!=j:
                    dx=-self.x[i,:]+self.x[j,:]
                    drsqr=np.sum(dx**2)
                    
                    if drsqr<soft**2:
                        drsqr = soft**2
                
                    r3=1/(drsqr*np.sqrt(drsqr))
                    
                    self.f[i,:]=self.f[i,:]+dx*self.m[j]*r3
        
    #this is for calculting forces with position values (xx) that are not ever saved
    def get_forces_2(self, xx, soft=0.01, do_pot=False):
        self.f[:]=0
        for i in range(self.n):
            for j in range(self.n):
                if i!=j:
                    dx=-xx[i,:]+xx[j,:]
                    drsqr=np.sum(dx**2)
                    
                    if drsqr<soft**2:
                        drsqr = soft**2
                
                    r3=1/(drsqr*np.sqrt(drsqr))
                    self.f[i,:]=self.f[i,:]+dx*self.m[j]*r3

    def get_acc(self, soft=0.01, do_pot=False):
        self.a[:]=0
        for i in range(self.n):
            for j in range(self.n):
                if i!=j:
                    dx=-self.x[i,:]+self.x[j,:]
                    drsqr=np.sum(dx**2)
                    
                    if drsqr<soft**2:
                        drsqr = soft**2
                
                    r3=1/(drsqr*np.sqrt(drsqr))
                    
                    self.a[i,:]=self.a[i,:]+dx*r3 #going to be the same really as F as m=1 rn


    def take_step_leapfrog(self, dt, soft=0.01):
        self.get_acc()
        v_h = self.v + self.a*dt
        self.x = self.x + v_h*dt
        self.v = v_h #so the V saved will be a half step behind x
        
    #############################################
    
    ############ methods for calculations with convolution (for project) #############
    

    """
    acc = grad phi

    calculate the potential using the Green's function!
    So you can use the Green's function of the laplacian : -1/(4*pi*r) where r = sqrt(x^2 + y^2 + z^2)
    Since (taking 4*pi*G=1): laplacian(phi) = rho ,

    solve phi = ifft(fft(G_laplacian) * fft(rho))  where G_laplacian is just the Green's function of the laplacian
    """
    
    def r_grid_from_point(self, point):
        t = np.zeros( (len(self.grid[0]), len(self.grid[1]), len(self.grid[2])) ) 
         
        # could also just do this, but I want my point in the center so i don't lose most of it cause that seems 
        # problematic - well this works as long as the box is a cube
        grid_arr = np.array(self.grid)
        t = np.sum((grid_arr + (self.dt)/2.0 -point[0])**2,axis=0)

        return t
    
    def G_laplacian(self, soften=0.01):
        #can we just use a non-real particle at the center to make this easier? Yes i think so
        
        centerPoint = [self.grid_shape[0]/2.0, self.grid_shape[1]/2.0, self.grid_shape[2]/2.0]
        R_arr = self.r_grid_from_point(centerPoint)
        R_arr[R_arr<soften] = soften
        
        green = -1/(4*np.pi*R_arr)
        
        green += np.flip(green,0)
        green += np.flip(green,1)
        green += np.flip(green,2)
        
        return green
    
    def get_density(self):
        
        #use a histogram to do way easier
        
        #get our bin edges - need one extra point compared to our grid as we need all the edges
        dt = self.dt
        x_g = np.arange(0, len(self.grid[0])*dt + dt, dt) #these can't deal with particles in -ve space -> BC will deal with this?
        y_g = np.arange(0, len(self.grid[1])*dt + dt, dt)
        z_g = np.arange(0, len(self.grid[2])*dt + dt, dt)
        
        #now fill the histgram, with the masses as the weights
        Hist, edges = np.histogramdd(self.x, bins=[x_g,y_g,y_g], weights=self.m)
            
        #get total mass per grid cube, then just divide by the volume of each cube
        density = Hist/dt**3
        return density
     
    def get_forces_conv(self, soft=0.01, do_pot=False):
        rho = self.get_density() #I think this looks decent
    
        green = self.G_laplacian()
#         phi = convFunction(green, rho, p=5)  
        phi = convFunction2(green, rho, dim=len(self.x[0]))  
        self.potential = phi
        
#         really f = grad phi
        #grad - needs BC applied    
        if self.BC:
    
            #just df/dx = (f(x+dt)-f(x-dt))/2dt for now - as this rolls over really it's for periodic
            grad_x = (np.roll(phi,1,axis=0)-np.roll(phi,-1,axis=0))/(2*self.dt)
            grad_y = (np.roll(phi,1,axis=1)-np.roll(phi,-1,axis=1))/(2*self.dt)
            grad_z = (np.roll(phi,1,axis=2)-np.roll(phi,-1,axis=2))/(2*self.dt) 
            
        else: #need to add 0 padding outside and then after grad get correct size back
            #just df/dx = (f(x+dt)-f(x-dt))/2dt for now
#             numpy.pad(phi, [1,1])
            
            grad_x = (np.roll(phi,1,axis=0)-np.roll(phi,-1,axis=0))/(2*self.dt)
            grad_y = (np.roll(phi,1,axis=1)-np.roll(phi,-1,axis=1))/(2*self.dt)
            grad_z = (np.roll(phi,1,axis=2)-np.roll(phi,-1,axis=2))/(2*self.dt) 
            
            #re-size somehow
       
        grad_x = -rho*grad_x
        grad_y = -rho*grad_y
        grad_z = -rho*grad_z
    
        #should stack all the forces so this is 4d -> [f_x, f_y, f_z] for each grid cell
        self.f_grid = np.stack((grad_x, grad_y, grad_z), axis=3) 
        
        return phi, self.f_grid, grad_x, grad_y, grad_z


    def get_acc_conv(self, soft=0.01, do_pot=False):
        #first calculate the force in each grid cell
        self.get_forces_conv()
        out = 0
        
        for i in range(self.n):
            #need to figure out which cell each particle is in
            #then pick out the force in that cell to use here
            
#             print("position? ", self.x[i,:])
            
            cell_n = np.floor(self.x[i,:]/(self.dt)).astype('int64') 
            #got to watch out when particles leave our cube and deal with them
            if np.any(cell_n < 0) or np.any(cell_n >= len(self.grid[0])): #could I just use a mod for periodic
#                 print("particle out!")
                out += 1
                
                if self.BC: #could I just use a mod for periodic?
                    cell_n = np.floor(self.x[i,:]/(self.dt)).astype('int64') % len(self.grid[0])
                    
                else:
                    cell_n = [0,0,0]
                    self.m[i] = 0.0000001
            
            f = self.f_grid[cell_n[0]][cell_n[1]][cell_n[2]]
            
#             print("f, cell#:", f, cell_n)
            
            self.a[i,:]=f/self.m[i]
        print("particles out from acc matching:", out)
            
        return self.a
    
    def check_BC(self):
        
        out = 0
        
        for i in range(self.n):           
#             print("position? ", self.x[i,:])
            
            cell_n = np.floor(self.x[i,:]/(self.dt)).astype('int64') 
            #got to watch out when particles leave our cube and deal with them
            
            if np.any(cell_n < 0) or np.any(cell_n >= len(self.grid[0])): #could I just use a mod for periodic
#                 print("particle out!")
                out += 1
                
                if self.BC: #could I just use a mod for periodic?
                    self.x[i,:] = self.x[i,:] % len(self.grid[0]) #this will only go big to small side tho
                    
                else: #need to actually remove them somehow?
                    self.x[i,:] = [0,0,0]
                    self.m[i] = 0.0000001
                    
        print("particles out:", out)


    def take_step_leapfrog_conv(self, dt, soft=0.01):
        self.get_acc_conv()
        v_h = self.v + self.a*dt
        self.x = self.x + v_h*dt
        self.check_BC()
        
        self.v = v_h #so the V saved will be a half step behind x
        
    def get_energy(self):
        e_K = np.sum( 0.5*self.m*(np.sum(self.v**2, axis=1))**2)
        
        e_P = -0.5*np.sum(self.potential) #somehow use phi to get this?
        
        e_total = e_K + e_P
        return e_total
    
#helper fucntions
def convFunction(arr1, arr2, p=5): #is this actually working well? I don't think so

    arrFT1 = np.fft.fft(np.pad(arr1, [0, p]))
    arrFT2 = np.fft.fft(np.pad(arr2, [0, p]))
        
    convolved = np.fft.ifft(arrFT1 * arrFT2)/len(arr1)
    
    if p > 0:
        convolved = convolved[:-p,:-p,:-p]

    return convolved.real

def convFunction2(arr1, arr2, dim=3): #this guy works I think
 
    arrFT1 = np.fft.rfftn(arr1)
    arrFT2 = np.fft.rfftn(arr2)
    
    phi = np.fft.irfftn(arrFT1 * arrFT2, s=arr1.shape) # need to have s to get odd length arrays

    for i in range(dim):
            phi = 0.5*(np.roll(phi,1,axis=i)+phi)    
    return phi

In [5]:
#checking stuff
x = np.array([4,6,1])
dt=1
cell = np.floor(x/(dt)).astype('int64')
cell_n = np.floor(x/(dt)).astype('int64') % 5


print(cell, cell_n)

[4 6 1] [4 1 1]


In [22]:
n=10000 #runs really quite slow with 100 000 points
gridSize = [10, 10, 10]

#single particle at rest
x=np.random.uniform(0, gridSize[0],(n,3))
v=np.random.randn(n,3)*0 #at rest?

# print(x)

m=np.ones(n) #mass

dt_t = 1.

# parts=Particles(x,v,m, gridSize, grid_dx=dt_t)
parts=Particles(x,v,m, gridSize, grid_dx=dt_t, BC_per=False)

dt=0.01

x_g = np.arange(0, gridSize[0], dt_t)
y_g = np.arange(0, gridSize[1], dt_t)
z_g = np.arange(0, gridSize[2], dt_t)
grid = np.meshgrid(x_g, y_g, z_g)
xx,yy,zz = np.meshgrid(x_g, y_g, z_g)

# for i in range(0, int(gridSize[0]/dt_t)):
#     plt.pcolor(yy,zz, phi_test[i],  shading='auto')
# #     plt.pcolor(yy,zz, g[i],  shading='auto')
    
#     plt.title("x = {}".format(i*dt_t))
#     plt.xlabel("z")
#     plt.ylabel("y")
    
#     plt.pause(2)
# plt.colorbar()

# """

# https://colab.research.google.com/drive/1zc1nvra1HF01mTtpOlRigj-UBVzYgFqq#scrollTo=pvtJPFS3LfYv

fig = plt.figure()
ax = Axes3D(fig)
scatt = ax.scatter(parts.x[:,0], parts.x[:,1], parts.x[:,2], color="royalblue",marker=".")#,s=.1)
ax.set_xlabel("x",fontsize=14)
ax.set_ylabel("y",fontsize=14)
ax.set_zlabel("z",fontsize=14)
ax.set_title("Initial conditions of the Universe\n",fontsize=20)

ax.set_xlim3d(0, gridSize[0])
ax.set_ylim3d(0,gridSize[1])
ax.set_zlim3d(0,gridSize[2])

plt.pause(1.)

# """    
for k in range(50):
    for l in range(2):
        parts.take_step_leapfrog_conv(dt)
   
    scatt.remove() #clears prev points
    scatt = ax.scatter(parts.x[:,0], parts.x[:,1], parts.x[:,2], color="royalblue",marker=".", alpha=0.7)
#     ax.set_title("frame {}".format(k))
#     ax.text2D(0.05, 0.95, "frame {}".format(k), transform=ax.transAxes)
    print("E total:", parts.get_energy())
    plt.pause(0.1)
    
# """   

particles out from acc matching: 0
particles out: 12
particles out from acc matching: 0
particles out: 34
E total: 1.3751702206367442e+21
particles out from acc matching: 0
particles out: 61
particles out from acc matching: 0
particles out: 104
E total: 1.0953194127756506e+23
particles out from acc matching: 0
particles out: 159
particles out from acc matching: 0
particles out: 205
E total: 1.3359737234009047e+24
particles out from acc matching: 0
particles out: 262
particles out from acc matching: 0
particles out: 335
E total: 1.757640329153653e+25
particles out from acc matching: 0
particles out: 411
particles out from acc matching: 0
particles out: 505
E total: 3.060183132259179e+26
particles out from acc matching: 0
particles out: 630
particles out from acc matching: 0
particles out: 748
E total: 3.847788096017847e+27
particles out from acc matching: 0
particles out: 914
particles out from acc matching: 0
particles out: 1124
E total: 1.3602811947176834e+28
particles out from acc ma

KeyboardInterrupt: 

### Part 1: 
Using this code, show that a single particle starting at rest remains motionless. 

In [17]:
n=1
#single particle at rest
x=np.abs(np.random.randn(n,3))
v=np.random.randn(n,3)*0 #at rest

print(x, v)

m=np.ones(n) #mass

gridSize = [5, 5, 5]
dt_t = 1.

parts=Particles(x,v,m, gridSize, grid_dx=dt_t)

dt=0.01

colors = cm.rainbow(np.linspace(0, 1, n))

fig = plt.figure()
ax = Axes3D(fig)    

for k in range(1000):
    for l in range(20):
#         parts.take_step_leapfrog(dt)
        parts.take_step_leapfrog_conv(dt)
        
#     print('step ',k)
    
#     plt.clf() #clears prev graphs
    for n_i in range(n):
        ax.scatter(parts.x[n_i][0], parts.x[n_i][1], parts.x[n_i][2], color=colors[n_i])
    print("E total:", parts.get_energy())
    ax.set_title("frame {}".format(k))
    plt.pause(0.01)


[[1.3254149  0.15115745 1.28979501]] [[-0. -0. -0.]]
particles out from acc matching: 0
particles out: 0
particles out from acc matching: 0
particles out: 0
particles out from acc matching: 0
particles out: 0
particles out from acc matching: 0
particles out: 0
particles out from acc matching: 0
particles out: 0
particles out from acc matching: 0
particles out: 0
particles out from acc matching: 0
particles out: 0
particles out from acc matching: 0
particles out: 0
particles out from acc matching: 0
particles out: 0
particles out from acc matching: 0
particles out: 0
particles out from acc matching: 0
particles out: 0
particles out from acc matching: 0
particles out: 0
particles out from acc matching: 0
particles out: 0
particles out from acc matching: 0
particles out: 0
particles out from acc matching: 0
particles out: 0
particles out from acc matching: 0
particles out: 0
particles out from acc matching: 0
particles out: 0
particles out from acc matching: 0
particles out: 0
particles o

particles out from acc matching: 0
particles out: 0
particles out from acc matching: 0
particles out: 0
particles out from acc matching: 0
particles out: 0
particles out from acc matching: 0
particles out: 0
particles out from acc matching: 0
particles out: 0
particles out from acc matching: 0
particles out: 0
particles out from acc matching: 0
particles out: 0
particles out from acc matching: 0
particles out: 0
particles out from acc matching: 0
particles out: 0
particles out from acc matching: 0
particles out: 0
particles out from acc matching: 0
particles out: 0
particles out from acc matching: 0
particles out: 0
particles out from acc matching: 0
particles out: 0
particles out from acc matching: 0
particles out: 0
particles out from acc matching: 0
particles out: 0
particles out from acc matching: 0
particles out: 0
particles out from acc matching: 0
particles out: 0
particles out from acc matching: 0
particles out: 0
particles out from acc matching: 0
particles out: 0
particles ou

particles out from acc matching: 0
particles out: 0
particles out from acc matching: 0
particles out: 0
particles out from acc matching: 0
particles out: 0
particles out from acc matching: 0
particles out: 0
particles out from acc matching: 0
particles out: 0
particles out from acc matching: 0
particles out: 0
particles out from acc matching: 0
particles out: 0
particles out from acc matching: 0
particles out: 0
particles out from acc matching: 0
particles out: 0
particles out from acc matching: 0
particles out: 0
particles out from acc matching: 0
particles out: 0
particles out from acc matching: 0
particles out: 0
particles out from acc matching: 0
particles out: 0
particles out from acc matching: 0
particles out: 0
particles out from acc matching: 0
particles out: 0
particles out from acc matching: 0
particles out: 0
particles out from acc matching: 0
particles out: 0
particles out from acc matching: 0
particles out: 0
particles out from acc matching: 0
particles out: 0
particles ou

particles out: 0
particles out from acc matching: 0
particles out: 0
particles out from acc matching: 0
particles out: 0
particles out from acc matching: 0
particles out: 0
E total: 41.31662322665603
particles out from acc matching: 0
particles out: 0
particles out from acc matching: 0
particles out: 0
particles out from acc matching: 0
particles out: 0
particles out from acc matching: 0
particles out: 0
particles out from acc matching: 0
particles out: 0
particles out from acc matching: 0
particles out: 0
particles out from acc matching: 0
particles out: 0
particles out from acc matching: 0
particles out: 0
particles out from acc matching: 0
particles out: 0
particles out from acc matching: 0
particles out: 0
particles out from acc matching: 0
particles out: 0
particles out from acc matching: 0
particles out: 0
particles out from acc matching: 0
particles out: 0
particles out from acc matching: 0
particles out: 0
particles out from acc matching: 0
particles out: 0
particles out from a

particles out: 0
particles out from acc matching: 0
particles out: 0
particles out from acc matching: 0
particles out: 0
particles out from acc matching: 0
particles out: 0
particles out from acc matching: 0
particles out: 0
particles out from acc matching: 0
particles out: 0
particles out from acc matching: 0
particles out: 0
particles out from acc matching: 0
particles out: 0
particles out from acc matching: 0
particles out: 0
particles out from acc matching: 0
particles out: 0
particles out from acc matching: 0
particles out: 0
particles out from acc matching: 0
particles out: 0
particles out from acc matching: 0
particles out: 0
particles out from acc matching: 0
particles out: 0
E total: 41.31662322665603
particles out from acc matching: 0
particles out: 0
particles out from acc matching: 0
particles out: 0
particles out from acc matching: 0
particles out: 0
particles out from acc matching: 0
particles out: 0
particles out from acc matching: 0
particles out: 0
particles out from a

particles out from acc matching: 0
particles out: 0
particles out from acc matching: 0
particles out: 0
particles out from acc matching: 0
particles out: 0
particles out from acc matching: 0
particles out: 0
particles out from acc matching: 0
particles out: 0
particles out from acc matching: 0
particles out: 0
particles out from acc matching: 0
particles out: 0
particles out from acc matching: 0
particles out: 0
particles out from acc matching: 0
particles out: 0
particles out from acc matching: 0
particles out: 0
particles out from acc matching: 0
particles out: 0
particles out from acc matching: 0
particles out: 0
particles out from acc matching: 0
particles out: 0
particles out from acc matching: 0
particles out: 0
particles out from acc matching: 0
particles out: 0
particles out from acc matching: 0
particles out: 0
particles out from acc matching: 0
particles out: 0
particles out from acc matching: 0
particles out: 0
particles out from acc matching: 0
particles out: 0
E total: 41.

particles out from acc matching: 0
particles out: 0
particles out from acc matching: 0
particles out: 0
particles out from acc matching: 0
particles out: 0
particles out from acc matching: 0
particles out: 0
particles out from acc matching: 0
particles out: 0
particles out from acc matching: 0
particles out: 0
particles out from acc matching: 0
particles out: 0
particles out from acc matching: 0
particles out: 0
particles out from acc matching: 0
particles out: 0
particles out from acc matching: 0
particles out: 0
particles out from acc matching: 0
particles out: 0
particles out from acc matching: 0
particles out: 0
particles out from acc matching: 0
particles out: 0
particles out from acc matching: 0
particles out: 0
particles out from acc matching: 0
particles out: 0
particles out from acc matching: 0
particles out: 0
particles out from acc matching: 0
particles out: 0
particles out from acc matching: 0
particles out: 0
particles out from acc matching: 0
particles out: 0
particles ou

particles out from acc matching: 0
particles out: 0
particles out from acc matching: 0
particles out: 0
particles out from acc matching: 0
particles out: 0
particles out from acc matching: 0
particles out: 0
particles out from acc matching: 0
particles out: 0
particles out from acc matching: 0
particles out: 0
particles out from acc matching: 0
particles out: 0
particles out from acc matching: 0
particles out: 0
particles out from acc matching: 0
particles out: 0
particles out from acc matching: 0
particles out: 0
particles out from acc matching: 0
particles out: 0
particles out from acc matching: 0
particles out: 0
E total: 41.31662322665603
particles out from acc matching: 0
particles out: 0
particles out from acc matching: 0
particles out: 0
particles out from acc matching: 0
particles out: 0
particles out from acc matching: 0
particles out: 0
particles out from acc matching: 0
particles out: 0
particles out from acc matching: 0
particles out: 0
particles out from acc matching: 0
pa

particles out from acc matching: 0
particles out: 0
particles out from acc matching: 0
particles out: 0
particles out from acc matching: 0
particles out: 0
particles out from acc matching: 0
particles out: 0
particles out from acc matching: 0
particles out: 0
particles out from acc matching: 0
particles out: 0
particles out from acc matching: 0
particles out: 0
particles out from acc matching: 0
particles out: 0
particles out from acc matching: 0
particles out: 0
particles out from acc matching: 0
particles out: 0
particles out from acc matching: 0
particles out: 0
particles out from acc matching: 0
particles out: 0
E total: 41.31662322665603


KeyboardInterrupt: 

### Part 2: 
Next, show that a pair of particles placed in a circular orbit continue to orbit each other, for at least some reasonable length of time. 

In [19]:
n=2
#two particles in circular orbit

gridSize = [10, 10, 10]

x=np.zeros([n,3])
v=np.zeros([n,3])
x[0,0]=gridSize[0]/2
x[0,1]=gridSize[1]/2
x[0,2]=gridSize[2]/2 - 1

x[1,0]=gridSize[0]/2
x[1,1]=gridSize[1]/2
x[1,2]=gridSize[2]/2 + 1

# v[0,1]=0.25
# v[1,1]=-0.25
# v[0,2]=0.25
# v[1,2]=-0.25

v[0,0]=0.1
v[0,1]=0.1

v[1,0]=-.1
v[1,1]=-.1

m=np.ones(n)*10 #mass

dt_t = 0.5#1.0
parts=Particles(x,v,m, gridSize, grid_dx=dt_t)

dt=0.05

colors = cm.rainbow(np.linspace(0, 1, n)) 

fig = plt.figure()
ax = Axes3D(fig)
        
for k in range(100):
    for l in range(1):
        parts.take_step_leapfrog_conv(dt)
    
#     print('step ',k)

    #     plt.clf() #clears prev graphs
    for n_i in range(n):
        if k==0:
            ax.scatter(parts.x[n_i][0], parts.x[n_i][1], parts.x[n_i][2])#, color=colors[n_i])
            
        else:
            ax.scatter(parts.x[n_i][0], parts.x[n_i][1], parts.x[n_i][2], color=colors[n_i])
#     ax.set_title("frame {}".format(k))
#     ax.text2D(0.05, 0.95, "frame {}".format(k), transform=ax.transAxes)
    plt.pause(0.01)

particles out from acc matching: 0
particles out: 0
particles out from acc matching: 0
particles out: 0
particles out from acc matching: 0
particles out: 0
particles out from acc matching: 0
particles out: 0
particles out from acc matching: 0
particles out: 0
particles out from acc matching: 0
particles out: 0
particles out from acc matching: 0
particles out: 0
particles out from acc matching: 0
particles out: 0
particles out from acc matching: 0
particles out: 0
particles out from acc matching: 0
particles out: 0
particles out from acc matching: 0
particles out: 0
particles out from acc matching: 0
particles out: 0
particles out from acc matching: 0
particles out: 0
particles out from acc matching: 0
particles out: 0
particles out from acc matching: 0
particles out: 0
particles out from acc matching: 0
particles out: 0
particles out from acc matching: 0
particles out: 0
particles out from acc matching: 0
particles out: 0
particles out from acc matching: 0
particles out: 0
particles ou

KeyboardInterrupt: 

In [ ]:
"""
Makes a cool pattern, but not a circle
x[0,0]=2.3 #p1 x
x[0,1]=2.5 #p1 y
x[0,2]=2.5 #p1 z

x[1,0]=2.8 #p2 x
x[1,1]=2.5 #p2 y
x[1,2]=2.5 #p2 z

v[0,0]=0.3
v[0,1]=0.3

v[1,0]=-0.3
v[1,1]=-0.3

gridSize = np.array([5, 5, 5])

#     positions = np.array([[0.5,0.5,0.5],[0.5,0.6,0.6]])
#     mass = np.ones(2)*5
#     vel = np.array([[0.04,0.04,0], [-0.04,-0.04, 0]])

m=np.ones(n)*8 #mass

dt_t = .5#1.0"""

"""
This is very, very close to a circle - without green flip thing
x=np.zeros([n,3])
v=np.zeros([n,3])
x[0,0]=2.4 #p1 x
x[0,1]=2.5 #p1 y
x[0,2]=2.5 #p1 z

x[1,0]=2.6 #p2 x
x[1,1]=2.5 #p2 y
x[1,2]=2.5 #p2 z

# vel = np.sqrt(5/(2*np.abs(x[0,0]-x[1,0])))
vel = 0.3


v[0,0]=0
v[0,1]=vel

v[1,0]=0
v[1,1]=-vel

gridSize = np.array([5, 5, 5])

m=np.ones(n)*5 #mass

dt_t = 0.5#1.0
parts=Particles(x,v,m, gridSize, grid_dx=dt_t)
colors = cm.rainbow(np.linspace(0, 1, n))

dt=0.01
"""

In [20]:
n=2 #so this works for 1 particle, but only for certain dt/gridsize values 

x=np.zeros([n,3])
v=np.zeros([n,3])
x[0,0]=2.0 #p1 x
x[0,1]=2.5 #p1 y
x[0,2]=2.5 #p1 z

x[1,0]=3.0 #p2 x
x[1,1]=2.5 #p2 y
x[1,2]=2.5 #p2 z

# vel = np.sqrt(5/(2*np.abs(x[0,0]-x[1,0])))
vel = .55

v[0,0]=vel
v[0,1]=vel

v[1,0]=-vel
v[1,1]=-vel

gridSize = np.array([5, 5, 5])

m=np.ones(n)*1 #mass

dt_t = 0.5#1.0
parts=Particles(x,v,m, gridSize, grid_dx=dt_t)
colors = cm.rainbow(np.linspace(0, 1, n))

dt=0.01

# fig = plt.figure()
# ax = Axes3D(fig)
# ax.scatter(parts.x[n_i][0], parts.x[n_i][1], parts.x[n_i][2], color=colors[n_i])
# plt.pause(0.01)
# plt.show()

#looking at the density to see if it makes sense
"""
d = parts.get_density()

x=np.arange(0, gridSize[0], dt_t)
yy,zz=np.meshgrid(x,x)

for i in range(0, int(gridSize[0]/dt_t)):
    plt.pcolor(yy,zz, d[i],  shading='auto')
    plt.title("x = {}".format(i*dt_t))
    plt.xlabel("z")
    plt.ylabel("y")
    
    plt.pause(2)
plt.colorbar()
print(d)
"""

phi_test, f_test, grad_x, grad_y, grad_z = parts.get_forces_conv()
g = parts.G_laplacian()

# x=np.arange(0, gridSize[0], dt_t)
# xx,yy,zz=np.meshgrid(x,x,xs)

x_g = np.arange(0, gridSize[0], dt_t)
y_g = np.arange(0, gridSize[1], dt_t)
z_g = np.arange(0, gridSize[2], dt_t)
grid = np.meshgrid(x_g, y_g, z_g)
xx,yy,zz = np.meshgrid(x_g, y_g, z_g)


# for i in range(0, int(gridSize[0]/dt_t)):
#     plt.pcolor(yy,zz, f_test[i],  shading='auto')
#     plt.title("x = {}".format(i*dt_t))
#     plt.xlabel("z")
#     plt.ylabel("y")
    
#     plt.pause(2)
# plt.colorbar()

# print(grid[0].shape, grad_x.shape)

# print(grad_x)

# fig = plt.figure()
# ax = fig.gca(projection='3d')
# for n_i in range(n):
#     ax.scatter(parts.x[n_i][0], parts.x[n_i][1], parts.x[n_i][2], color=colors[n_i])
# ax.quiver(xx, yy, zz, grad_x, grad_y, grad_z)#, length=1.0)
# plt.pause(5)

x=np.arange(0, gridSize[0], dt_t)
yy,zz=np.meshgrid(x,x)

# for i in range(0, int(gridSize[0]/dt_t)):
#     plt.pcolor(yy,zz, phi_test[i],  shading='auto')
# #     plt.pcolor(yy,zz, g[i],  shading='auto')
    
#     plt.title("x = {}".format(i*dt_t))
#     plt.xlabel("z")
#     plt.ylabel("y")
    
#     plt.pause(2)
# plt.colorbar()

# """
colors = cm.rainbow(np.linspace(0, 1, n)) 

fig = plt.figure()
ax = Axes3D(fig)
ax.set_xlabel('x')
ax.set_ylabel('y')
ax.set_zlabel('z')
        
for k in range(300):
    for l in range(4):
        parts.take_step_leapfrog_conv(dt)
    
    print('step ',k)

    #     plt.clf() #clears prev graphs
    for n_i in range(n):
        if k==0:
            ax.scatter(parts.x[n_i][0], parts.x[n_i][1], parts.x[n_i][2])#, color=colors[n_i])
        else:
            ax.scatter(parts.x[n_i][0], parts.x[n_i][1], parts.x[n_i][2], color=colors[n_i])
#     ax.set_title("frame {}".format(k))
#     ax.text2D(0.05, 0.95, "frame {}".format(k), transform=ax.transAxes)
    print("E total:", parts.get_energy())

    plt.pause(0.01)
# """   
l=8

particles out from acc matching: 0
particles out: 0
particles out from acc matching: 0
particles out: 0
particles out from acc matching: 0
particles out: 0
particles out from acc matching: 0
particles out: 0
step  0
E total: 1455.4262811426763
particles out from acc matching: 0
particles out: 0
particles out from acc matching: 0
particles out: 0
particles out from acc matching: 0
particles out: 0
particles out from acc matching: 0
particles out: 0
step  1
E total: 1455.4319366669338
particles out from acc matching: 0
particles out: 0
particles out from acc matching: 0
particles out: 0
particles out from acc matching: 0
particles out: 0
particles out from acc matching: 0
particles out: 0
step  2
E total: 1455.4417814784324
particles out from acc matching: 0
particles out: 0
particles out from acc matching: 0
particles out: 0
particles out from acc matching: 0
particles out: 0
particles out from acc matching: 0
particles out: 0
step  3
E total: 1455.4559726675577
particles out from acc m

particles out from acc matching: 0
particles out: 0
particles out from acc matching: 0
particles out: 0
particles out from acc matching: 0
particles out: 0
particles out from acc matching: 0
particles out: 0
step  34
E total: 1455.267547144825
particles out from acc matching: 0
particles out: 0
particles out from acc matching: 0
particles out: 0
particles out from acc matching: 0
particles out: 0
particles out from acc matching: 0
particles out: 0
step  35
E total: 1455.2920434406128
particles out from acc matching: 0
particles out: 0
particles out from acc matching: 0
particles out: 0
particles out from acc matching: 0
particles out: 0
particles out from acc matching: 0
particles out: 0
step  36
E total: 1455.324895196374
particles out from acc matching: 0
particles out: 0
particles out from acc matching: 0
particles out: 0
particles out from acc matching: 0
particles out: 0
particles out from acc matching: 0
particles out: 0
step  37
E total: 1455.3674084756908
particles out from acc

particles out from acc matching: 0
particles out: 0
particles out from acc matching: 0
particles out: 0
particles out from acc matching: 0
particles out: 0
particles out from acc matching: 0
particles out: 0
step  68
E total: 1455.3385315109022
particles out from acc matching: 0
particles out: 0
particles out from acc matching: 0
particles out: 0
particles out from acc matching: 0
particles out: 0
particles out from acc matching: 0
particles out: 0
step  69
E total: 1455.3481304142251
particles out from acc matching: 0
particles out: 0
particles out from acc matching: 0
particles out: 0
particles out from acc matching: 0
particles out: 0
particles out from acc matching: 0
particles out: 0
step  70
E total: 1455.3615848101083
particles out from acc matching: 0
particles out: 0
particles out from acc matching: 0
particles out: 0
particles out from acc matching: 0
particles out: 0
particles out from acc matching: 0
particles out: 0
step  71
E total: 1455.3791364185251
particles out from a

particles out from acc matching: 0
particles out: 0
particles out from acc matching: 0
particles out: 0
particles out from acc matching: 0
particles out: 0
particles out from acc matching: 0
particles out: 0
step  102
E total: 1455.355989051544
particles out from acc matching: 0
particles out: 0
particles out from acc matching: 0
particles out: 0
particles out from acc matching: 0
particles out: 0
particles out from acc matching: 0
particles out: 0
step  103
E total: 1455.394367170535
particles out from acc matching: 0
particles out: 0
particles out from acc matching: 0
particles out: 0
particles out from acc matching: 0
particles out: 0
particles out from acc matching: 0
particles out: 0
step  104
E total: 1455.4026437302507
particles out from acc matching: 0
particles out: 0
particles out from acc matching: 0
particles out: 0
particles out from acc matching: 0
particles out: 0
particles out from acc matching: 0
particles out: 0
step  105
E total: 1455.3073821824999
particles out from

particles out from acc matching: 0
particles out: 0
particles out from acc matching: 0
particles out: 0
particles out from acc matching: 0
particles out: 0
step  138
E total: 1455.3588257687607
particles out from acc matching: 0
particles out: 0
particles out from acc matching: 0
particles out: 0
particles out from acc matching: 0
particles out: 0
particles out from acc matching: 0
particles out: 0
step  139
E total: 1455.376873581773
particles out from acc matching: 0
particles out: 0
particles out from acc matching: 0
particles out: 0
particles out from acc matching: 0
particles out: 0
particles out from acc matching: 0
particles out: 0
step  140
E total: 1455.3993479318915
particles out from acc matching: 0
particles out: 0
particles out from acc matching: 0
particles out: 0
particles out from acc matching: 0
particles out: 0
particles out from acc matching: 0
particles out: 0
step  141
E total: 1455.4266323485629
particles out from acc matching: 0
particles out: 0
particles out fro

KeyboardInterrupt: 

### Part 3: 
Set up both periodic and non-periodic boundary conditions. Set up a problem where hundreds of thousands of particles are initially scattered randomly throughout the domain. Show the evolution with time for both periodic and non-periodic boundary conditions. Track the total energy how well is it conserved? 

In [26]:
#no intial velocity?

n=10000
gridSize = [10, 10, 10]

#single particle at rest
x=np.random.uniform(0, gridSize[0],(n,3))
v=np.random.randn(n,3)*0 #at rest?

# print(x)

m=np.ones(n) #mass

dt_t = 1.

parts=Particles(x,v,m, gridSize, grid_dx=dt_t)

dt=0.01


In [29]:
# https://colab.research.google.com/drive/1zc1nvra1HF01mTtpOlRigj-UBVzYgFqq#scrollTo=pvtJPFS3LfYv
# print(parts.x[:,0])

fig = plt.figure()
ax = Axes3D(fig)
scatt = ax.scatter(parts.x[:,0], parts.x[:,1], parts.x[:,2], color="royalblue",marker=".")#,s=.1)
ax.set_xlabel("x",fontsize=14)
ax.set_ylabel("y",fontsize=14)
ax.set_zlabel("z",fontsize=14)
ax.set_title("Initial conditions of the Universe\n",fontsize=20)

ax.set_xlim3d(0, gridSize[0])
ax.set_ylim3d(0,gridSize[1])
ax.set_zlim3d(0,gridSize[2])

plt.pause(0.1)

# """    
for k in range(50):
    for l in range(2):
        parts.take_step_leapfrog_conv(dt)
   
    scatt.remove() #clears prev points
    scatt = ax.scatter(parts.x[:,0], parts.x[:,1], parts.x[:,2], color="royalblue",marker=".", alpha=0.7)
#     ax.set_title("frame {}".format(k))
#     ax.text2D(0.05, 0.95, "frame {}".format(k), transform=ax.transAxes)
    print("E total:", parts.get_energy())
    plt.pause(0.1)

particles out from acc matching: 0
particles out: 150
particles out from acc matching: 0
particles out: 206
E total: 7.016462414415736e+24
particles out from acc matching: 0
particles out: 290
particles out from acc matching: 0
particles out: 373
E total: 4.6580121991074375e+25
particles out from acc matching: 0
particles out: 478
particles out from acc matching: 0
particles out: 585
E total: 6.040097014368894e+26
particles out from acc matching: 0
particles out: 727
particles out from acc matching: 0
particles out: 904
E total: 1.5633146640460178e+27
particles out from acc matching: 0
particles out: 1132
particles out from acc matching: 0
particles out: 1412
E total: 2.2808806212208533e+27
particles out from acc matching: 0
particles out: 1720
particles out from acc matching: 0
particles out: 2104
E total: 2.2810708010441082e+27
particles out from acc matching: 0
particles out: 2489
particles out from acc matching: 0
particles out: 2829
E total: 2.2814054041452094e+27
particles out fr

KeyboardInterrupt: 

### Part 4: 
In cosmology, we start the universe with a scale-invariant power spectrum, so mass fluctuations are proportional to k−3. Start with the particles on a grid, but with masses derived from a realization of k−3 and use periodic boundary conditions (although not strictly necessary, you may with to start with your particles in the center of grid cells rather than at the corners). How does your universe look now?